In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/stockdata/new.csv
/kaggle/input/stockdata/top.csv
/kaggle/input/stockdata/all.csv
/kaggle/input/stockdata/Data/modIndex.csv
/kaggle/input/stockdata/Data/Index.csv
/kaggle/input/stockdata/Data/RiskFreeRate.csv
/kaggle/input/stockdata/Data/Equity.csv
/kaggle/input/stockdata/Data/Stock/fc500770.csv
/kaggle/input/stockdata/Data/Stock/532540.csv
/kaggle/input/stockdata/Data/Stock/fc500325.csv
/kaggle/input/stockdata/Data/Stock/fc532540.csv
/kaggle/input/stockdata/Data/Stock/533155.csv
/kaggle/input/stockdata/Data/Stock/500180.csv
/kaggle/input/stockdata/Data/Stock/fc500085.csv
/kaggle/input/stockdata/Data/Stock/fc533155.csv
/kaggle/input/stockdata/Data/Stock/fc533260.csv
/kaggle/input/stockdata/Data/Stock/gr500124.csv
/kaggle/input/stockdata/Data/Stock/gr500182.csv
/kaggle/input/stockdata/Data/Stock/gr500209.csv
/kaggle/input/stockdata/Data/Stock/fc532210.csv
/kaggle/input/stockdata/Data/Stock/532538.csv
/kaggle/input/stockdata/Data/Stock/fc500680.csv
/kaggle/input/stockdata/D

In [2]:
import os
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.backend import sigmoid
from sklearn.metrics import confusion_matrix
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import precision_score, make_scorer
import matplotlib.pyplot as plt
from math import sqrt
import threading
from multiprocessing.pool import ThreadPool
import time
import multiprocessing
import traceback
import tensorflow as tf

import prettytable

In [3]:
import warnings; warnings.simplefilter('ignore')

In [4]:
def pre_process_data(data,null_threshold):
    """
    Drops Date and Unix Date columns from the data.
    Drops the columns which has null values more than specified null_threshold.
    Replaces infinite values with NAN.
    Drops the rows which has null values.

    Parameters
    ----------
    data : dataframe

    null_threshold : numeric
        numeric value describing the amount of null values that can be present.

    Returns
    -------
    data : dataframe
        an updated dataframe after performing all the opertaions.
    """
    
    data.drop(columns=['Unix Date','Date'],axis=1,inplace=True)
    total = data.shape[0]
    for col in data.columns:
        if null_threshold * total / 100 < data[col].isnull().sum():
            data.drop(columns=[col],axis=1,inplace=True)
    data.replace([np.inf, -np.inf], np.nan, inplace=True)
    data = data.apply(pd.to_numeric,errors='coerce')
    data.dropna(axis=0,inplace=True)
    return data

In [5]:
def dependent_column(data,column):
    """
    Removes all the Next Day columns.
    Removes all the non Growth Rate Columns (GR)
    add the predictor column to list of columns.

    Parameters
    ----------
    data : dataframe

    column : string
        name of the predictor column 

    Returns
    -------
    data : dataframe
        an updated dataframe after performing all the opertaions.
    column : string
        name of the predictor column
    """
    cols = [col for col in data.columns if "next" not in col.lower() and col.lower().endswith("gr")]
    cols.append(column)
    data = data[cols]
    return (data,column)

In [6]:
def reshape_data_r(x_train, x_test, y_train, y_test,units = 100):
    my_x_train = list()
    my_y_train = list()
    my_x_test = list()
    my_y_test = list()
    for i in range(x_train.shape[0]-units):
        my_x_train.append(x_train.iloc[i:i+units,:])
        my_y_train.append(y_train.iloc[i+units,])
    
    my_x_train = np.array(my_x_train)
    my_x_train = np.reshape(my_x_train,(my_x_train.shape[0],my_x_train.shape[1],my_x_train.shape[2]))
    
    my_y_train = np.array(my_y_train)
    my_y_train = np.reshape(my_y_train,(my_y_train.shape[0],1))
    
    for i in range(x_test.shape[0]-units):
        my_x_test.append(x_test.iloc[i:i+units,:])
        my_y_test.append(y_test.iloc[i+units,])
        
    my_x_test = np.array(my_x_test)
    my_x_test = np.reshape(my_x_test,(my_x_test.shape[0],my_x_test.shape[1],my_x_test.shape[2]))
    
    my_y_test = np.array(my_y_test)
    my_y_test = np.reshape(my_y_test,(my_y_test.shape[0],1))
    
    return (my_x_train, my_x_test, my_y_train, my_y_test)

In [7]:
def build_lstm(input_shape):
    model = Sequential()
    model.add(LSTM(units=32,return_sequences=True,input_shape=input_shape))
    model.add(Dropout(0.2))
    model.add(LSTM(units=32,return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(units=32))
    model.add(Dropout(0.2))
    model.add(Dense(units = 1,activation = 'sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer="Adam", metrics=[tf.keras.metrics.Precision()])
    return model

In [8]:
def keras_model(input_dim,units = 32):
    activation='relu'
    dropout_rate=0.2
    init_mode='uniform'
    weight_constraint=0 
    optimizer='adam' 
    lr = 0.01
    momemntum=0
    
    model = Sequential()
    model.add(Dense(units = units, input_dim = input_dim, kernel_initializer=init_mode,activation=activation))
    model.add(Dropout(dropout_rate)) 
    model.add(Dense(units = units, input_dim = input_dim, kernel_initializer=init_mode,activation=activation))
    model.add(Dropout(dropout_rate)) 
    model.add(Dense(1, kernel_initializer=init_mode, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=[tf.keras.metrics.Precision()])

    return model

In [9]:
def best_parameters_rnn(X,Y):
    custom_scorer = make_scorer(precision_score, greater_is_better = True, pos_label = 1)
    batch_size = [25,32,48,64,100]
    epochs = [25,50,75,100]
    param_grid = dict(epochs=epochs,batch_size=batch_size)
    input_shape = (X.shape[1],X.shape[2])
    model = KerasClassifier(build_fn = build_lstm,input_shape=input_shape,verbose=1)
    grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, scoring=custom_scorer,verbose=1)
    grid_result = grid.fit(X,Y) 
    epoch = grid_result.best_params_['epochs']
    batch_size = grid_result.best_params_['batch_size']
    return (epoch,batch_size)

In [10]:
def create_confusion_matrix(y_pred,y_true):
    
    cm = confusion_matrix(y_true,y_pred)
    accuracy = metrics.accuracy_score(y_true,y_pred)
    precision = metrics.precision_score(y_true,y_pred)
    recall = metrics.recall_score(y_true,y_pred)
    f1_score = metrics.f1_score(y_true,y_pred)
    return {"accuracy":accuracy,"precision":precision,"recall":recall,"f1_score":f1_score,"confusion matrix":cm}

In [11]:
def error_metrics(y_true, y_pred):
    rmse = sqrt(metrics.mean_squared_error(y_true, y_pred))
    mae = metrics.mean_absolute_error(y_true, y_pred)
    mse = metrics.mean_squared_error(y_true, y_pred)
    return {"root_mean_squared_error":rmse,"mean_absolute_error":mae,"mean_squared_error":mse}

In [12]:
def split_dataset(X,Y,t):
    tr = int(len(X)*t)
    tt = len(X) - tr
    xtr = X[:tr]
    xtt = X[tr:tr+tt]
    ytr = Y[:tr]
    ytt = Y[tr:tr+tt]
    return (xtr,xtt,ytr,ytt)
    

In [13]:
def create_rnn_classification(df,column,epochs,batch_size,threshold):
    df["Target"] = df[column].apply(lambda x : 1 if x >= threshold else 0)
    X = df.drop(columns=["Target",column])
    Y = df["Target"]
    x_train, x_test, y_train, y_test = split_dataset(X,Y,0.70)
    x_train, x_test, y_train, y_test = reshape_data_r(x_train, x_test, y_train, y_test)
    input_dim = (x_train.shape[1],x_train.shape[2])
    
    model = KerasClassifier(build_fn = build_lstm, batch_size=batch_size, epochs=epochs,input_shape=input_dim,verbose=1) 

    history = model.fit(x_train,y_train,epochs = epochs,batch_size = batch_size, validation_data = (x_test,y_test),shuffle = False)

    y_pred = model.predict(x_test)
    y_pred = np.array(y_pred)
    y_pred = np.reshape(y_pred, (y_pred.shape[0],1))

    result = {}
    error = error_metrics(y_test, y_pred)
    confusion = create_confusion_matrix(y_test,y_pred)
    result.update(error)
    result.update(confusion)
    return result

In [14]:
def rnn_classification(df,company,column = "Next Day Close Price GR"):
    threshold = [0.01,0.02,0.03,0.04,0.05,0.1]
    solution = list()
    for t in threshold:
        df["Target"] = df[column].apply(lambda x : 1 if x >= t else 0)
        X = df.drop(columns=["Target",column])
        Y = df["Target"]
        name = str(company) + "-" + str(t) + ".csv"
        print(name)
        x_train, x_test, y_train, y_test = split_dataset(X,Y,0.70)
        x_train, x_test, y_train, y_test = reshape_data_r(x_train, x_test, y_train, y_test)
        epochs,batch_size = best_parameters_rnn(x_train,y_train)
        result = create_rnn_classification(df,column = "Next Day Close Price GR",epochs = epochs,batch_size = batch_size,threshold = t)
        result.update({"epochs":epochs,"batch_size":batch_size,"threshold":t})
        solution.append(result)
        try:
            pd.DataFrame([result]).to_csv(name)
        except:
            continue
    return solution

In [15]:
total = pd.read_csv("../input/stockdata/all.csv")

In [16]:
%%time
for _,row in total.iterrows():
    security_id = row['security id']
    try:
        print(security_id)
        df = pd.read_csv(os.path.join(inputpath,"gr"+str(security_id)+".csv"))
        df = pre_process_data(df,60)
        df,column = dependent_column(df, column = "Next Day Close Price GR")
        rnn_res = rnn_classification(df,security_id)
        rnn_df = pd.DataFrame(rnn_res)
        rnn_df.to_csv('rnn_'+str(security_id)+".csv",index=None)
    except Exception as e:
        traceback.print_exc() 

500112
500325
532540
500209
532174
507685
530965
500182
532210
500180
500680
506395
500770
500085
501425
532899
537291
500790
500825
533155
533287
533260
539921
542602
543217
543261
532538
500387
500425
532689
532706
532163
524715
532488
500124
CPU times: user 16 ms, sys: 7.01 ms, total: 23 ms
Wall time: 23.7 ms


Traceback (most recent call last):
  File "<timed exec>", line 5, in <module>
NameError: name 'inputpath' is not defined
Traceback (most recent call last):
  File "<timed exec>", line 5, in <module>
NameError: name 'inputpath' is not defined
Traceback (most recent call last):
  File "<timed exec>", line 5, in <module>
NameError: name 'inputpath' is not defined
Traceback (most recent call last):
  File "<timed exec>", line 5, in <module>
NameError: name 'inputpath' is not defined
Traceback (most recent call last):
  File "<timed exec>", line 5, in <module>
NameError: name 'inputpath' is not defined
Traceback (most recent call last):
  File "<timed exec>", line 5, in <module>
NameError: name 'inputpath' is not defined
Traceback (most recent call last):
  File "<timed exec>", line 5, in <module>
NameError: name 'inputpath' is not defined
Traceback (most recent call last):
  File "<timed exec>", line 5, in <module>
NameError: name 'inputpath' is not defined
Traceback (most recent call last